In [115]:
# Paths
strCsvFolderPath = "../Resources/CSVs/"
strDatabaseFolderPath = "../Resources/Databases/"
strJsonFolderPath = "../Resources/CSVs/"
strPasswdFolderPath = "../Resources/passwd/"

In [116]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from pprint import pprint

In [117]:
strUrl = "https://data.pr.gov/resource/gb92-58gc"
l_json = requests.get(strUrl).json()

# We create a temporal file for analysis
df_masterData = pd.DataFrame(l_json)
df_masterData.to_csv(strCsvFolderPath + "masterData.csv")

## Upload total population of Puerto Rico per year. Id = SP.POP.TOTL

In [118]:
strUrl = "http://api.worldbank.org/countries/PRI/indicators/SP.POP.TOTL?format=json&per_page=10000"
l_json = requests.get(strUrl).json()
l_json = l_json[1] 
# We create a temporal file for analysis
df_totalPopulation = pd.DataFrame(l_json)
df_totalPopulation.to_csv(strCsvFolderPath + "totalPopulationPerYears.csv")

## Population growth of Puerto Rico per year. Id = SP.POP.GROW

In [119]:
strUrl = "http://api.worldbank.org/countries/PRI/indicators/SP.POP.GROW?format=json&per_page=10000"
l_json = requests.get(strUrl).json()
l_json = l_json[1] 
# We create a temporal file for analysis
df_populationGrowth = pd.DataFrame(l_json)
df_populationGrowth.to_csv(strCsvFolderPath + "PopulationGrowthPerYears.csv")

## Age dependency ratio, young (% of working-age population). Id = SP.POP.DPND

In [120]:
strUrl = "http://api.worldbank.org/countries/PRI/indicators/SP.POP.DPND.YG?format=json&per_page=10000"
l_json = requests.get(strUrl).json()
l_json = l_json[1] 
# We create a temporal file for analysis
df_ageDependendency = pd.DataFrame(l_json)
df_ageDependendency.to_csv(strCsvFolderPath + "AgeDependencyRatio_Young_PerYears.csv")

## Function changeToCorrectStr

In [121]:
def changeToCorrectStr( pStr ):
    strErrorDict = {"a_o":"ñ", "l_s":"e", "b_s":"a", "m_t":"a"}
    for key in strErrorDict:
        pos = pStr.find(key)
        if pos != -1:
            ls = list(pStr)
            ls[pos+1] = strErrorDict[key]
            pStr = ''.join(str(cr) for cr in ls)
    return pStr

In [122]:
changeToCorrectStr('espa_ol_b_sico')

'español_basico'

## Function recoveryFmtDateToText

In [123]:
def recoveryFmtDateToText(pStr):
    monthOrder = {'Jan':1, 
                  'Feb':2, 
                  'Mar':3, 
                  'Apr':4, 
                  'May':5, 
                  'Jun':6, 
                  'Jul':7, 
                  'Aug':8, 
                  'Sep':9, 
                  'Oct':10, 
                  'Nov':11, 
                  'Dec':12 }
    l = pStr.split('-')
    return (str(monthOrder[l[1]]) + ' - ' + l[0])

In [124]:
recoveryFmtDateToText('9-Jul')

'7 - 9'

## First step: We create l_FilterHeaders with the correct DataFrame columns' names

In [125]:
strUrl = "https://data.pr.gov/resource/gb92-58gc"
l_jsonMasterData = requests.get(strUrl).json()
# We catch the first row from dataFrame df_masterData
dc_row = l_jsonMasterData[0]

l_FilterHeaders = []
for key in dc_row.keys():
    if key.find("grado_") == -1: # We filter only the columns whose name does not start with "grado_"
        newKey = changeToCorrectStr( key ) # Fix the name of the column
        l_FilterHeaders.append(newKey)
l_FilterHeaders

['bajo_nivel_pobreza',
 'codigo',
 'direccio_estado',
 'direccio_fisica',
 'direccion_municipio',
 'direccion_zipcode',
 'director',
 'distrito',
 'escuela',
 'español_avanzado',
 'español_basico',
 'español_pre_basico',
 'español_proficiente',
 'español_proficiente_avanzado',
 'estatus_actual',
 'fax',
 'geolocalizacion',
 'grados_original',
 'grados_posterior',
 'ingles_avanzado',
 'ingles_basico',
 'ingles_pre_basico',
 'ingles_proficiente',
 'ingles_proficiente_avanzado',
 'matematicas_avanzado',
 'matematicas_basico',
 'matematicas_pre_basico',
 'matematicas_proficiente',
 'matematicas_proficiente_avanzado',
 'matricula_total',
 'municipio_escolar',
 'nivel_original',
 'region',
 'table_id',
 'telefono',
 'zona']

## We clean the data and create the new columns latitude and longitude

In [126]:
l_newJson = [] # new Json with columns and their right values 
l_LatLong = [] # new two columns: Latitude and Longitude
for rec in l_jsonMasterData:
    # if the current record or row has less than 30 fields with data, then we discard it. 
    # if the record has more than 30 fields with data then we copy the important information to the list l_newJson
    if len(rec) > 30: 
        dc_temp = {}
        dc_LatLong = {}
        for key in rec.keys():
            newKey = changeToCorrectStr( key )
            if newKey in l_FilterHeaders: # We copy only the columns that do not start with the word grado_
                # If it is the "geolocalizacion" column we start copying the data from your dictionary to the list l_LatLong,
                # but we discard this data from the list l_newJson.
                if newKey == "geolocalizacion": 
                    dc_LatLong["latitude"] = rec["geolocalizacion"]["latitude"]
                    dc_LatLong["longitude"] = rec["geolocalizacion"]["longitude"] 
                    l_LatLong.append( dc_LatLong )
                else:
                    dc_temp[newKey]=rec[key]
        l_newJson.append( dc_temp )

## We verify that both "l_newJson" and "l_LatLong" have the same number of rows.

In [127]:
len( l_newJson )

961

In [128]:
len( l_LatLong )

961

## We use the function recoveryFmtDateToText, to fix, fields with date format.

In [129]:
l_months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
idx = 0
for rec in l_newJson:
    strVal1 = rec["grados_original"]
    strVal2 = rec["grados_posterior"]
    for month in l_months:
        if strVal1.find(month) != -1:
            strVal1 = recoveryFmtDateToText(strVal1)
            l_newJson[ idx ]["grados_original"] = strVal1
        if strVal2.find(month) != -1:
            strVal2 = recoveryFmtDateToText(strVal2)
            l_newJson[ idx ]["grados_posterior"] = strVal2
    idx = idx + 1

In [130]:
df_masterData = pd.DataFrame(l_newJson)
df_LatLong = pd.DataFrame(l_LatLong)

In [131]:
df_masterData.head()

,bajo_nivel_pobreza,codigo,direccio_estado,direccio_fisica,direccion_municipio,direccion_zipcode,director,distrito,escuela,español_avanzado,...,matematicas_pre_basico,matematicas_proficiente,matematicas_proficiente_avanzado,matricula_total,municipio_escolar,nivel_original,region,table_id,telefono,zona
0,86.24,35295,PR,CARR ESTATAL 200 ESQ CARR ESTATAL 993,VIEQUES,00765-1554,MYRIAM R RODRIGUEZ RODRIGUEZ,FAJARDO,20 DE SEPTIEMBRE DE 1988,6.88,...,45.87,1.38,1.38,218,VIEQUES,INTERMEDIO,HUMACAO,1,741-3091,URBANA
1,86.69,57323,PR,BO QUEBRADA CALLE 1 SECT CARACOLES,PENUELAS,00624-0000,GLORIA GARCIA,YAUCO,MIGUEL GONZALEZ BAUZA,15.03,...,17.65,16.99,29.41,280,PEÑUELAS,ELEMENTAL,PONCE,759,836-2597,RURAL
2,65.31,23218,PR,URB REPARTO MONTEBANCO CALLE D,CAYEY,00737-0000,WANDA I.CARTAGENA ORTIZ,CIDRA,MIGUEL MELENDEZ MUNOZ,32.84,...,10.45,21.89,50.25,278,CAYEY,ELEMENTAL,CAGUAS,760,738-2058,URBANA
3,65.71,23531,PR,AVE ANTONIO R BARCELO FRENTE CUC,CAYEY,00737-0000,PEDRO SANCHEZ,CIDRA,MIGUEL MELENDEZ MUNOZ,23.98,...,10.40,17.34,30.06,762,CAYEY,SUPERIOR,CAGUAS,761,738-3078,URBANA
4,63.10,70276,PR,CARR 840 KM 1 HM 1 SECTOR LA ALDEA,BAYAMON,00960-3169,MARIA JUDITH VAZQUEZ FONTANEZ,BAYAMON,MIGUEL MELENDEZ MUNOZ,44.59,...,19.59,16.89,35.14,577,BAYAMON II,SUPERIOR,BAYAMON,762,799-7300,RURAL


In [132]:
df_LatLong.head()

,latitude,longitude
0,18.14372,-65.44178
1,18.05554,-66.71168
2,18.1174,-66.14892
3,18.116,-66.16099
4,18.36999,-66.17751


## We insert the "latitude" and "longitude" columns in positions 16 and 17 of the df_masterData

In [133]:
df_masterData.insert(16,"latitude",df_LatLong[["latitude"]])
df_masterData.insert(17,"longitude",df_LatLong[["longitude"]])

In [134]:
df_masterData[["fax","latitude","longitude"]].head()

,fax,latitude,longitude
0,876-2804,18.14372,-65.44178
1,836-2597,18.05554,-66.71168
2,263-3044,18.1174,-66.14892
3,738-9065,18.116,-66.16099
4,799-6500,18.36999,-66.17751


## We verify that the values of the columns "grados_original" and "grados_posterior" have been fixed.

In [135]:
df_masterData[["grados_original","grados_posterior"]].head(20)

,grados_original,grados_posterior
0,"7 - 9 , EEI","7 - 9 , EEI"
1,"PK - 6 , EEE","PK - 6 , EEE"
2,"K - 6 , EEE","K - 6 , EEE"
3,10 - 12,10 - 12
4,10 - 12,10 - 12
5,"10 - 12 , EES","10 - 12 , EES"
6,10 - 12,10 - 12
7,"PK - 6 , EEE","PK - 6 , EEE"
8,"K - 6 , EEE","K - 8, EEE, EEI"
9,"K - 3 , EEE","K - 3 , EEE"


## Database Creation into SQLite

In [137]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base

strDataBaseName = "censusData.sqlite"

engine = create_engine(f"sqlite:///{strDatabaseFolderPath}{strDataBaseName}", echo=True)
Base = declarative_base()

Base.metadata.create_all(engine)

2019-05-18 03:26:13,878 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-05-18 03:26:13,879 INFO sqlalchemy.engine.base.Engine ()
2019-05-18 03:26:13,880 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-05-18 03:26:13,880 INFO sqlalchemy.engine.base.Engine ()


In [138]:
database_path = strDatabaseFolderPath + strDataBaseName
engine = create_engine(f'sqlite:///{database_path}')

In [139]:
engine.table_names()

[]

## Process of loading data.

In [140]:
df_masterData.to_sql(name='tblMasterData', con=engine, if_exists='append', index=False)

In [141]:
engine.table_names()

['tblMasterData']

In [144]:
pd.read_sql_query('select * from tblMasterData LIMIT 5', con=engine)

,bajo_nivel_pobreza,codigo,direccio_estado,direccio_fisica,direccion_municipio,direccion_zipcode,director,distrito,escuela,español_avanzado,...,matematicas_pre_basico,matematicas_proficiente,matematicas_proficiente_avanzado,matricula_total,municipio_escolar,nivel_original,region,table_id,telefono,zona
0,86.24,35295,PR,CARR ESTATAL 200 ESQ CARR ESTATAL 993,VIEQUES,00765-1554,MYRIAM R RODRIGUEZ RODRIGUEZ,FAJARDO,20 DE SEPTIEMBRE DE 1988,6.88,...,45.87,1.38,1.38,218,VIEQUES,INTERMEDIO,HUMACAO,1,741-3091,URBANA
1,86.69,57323,PR,BO QUEBRADA CALLE 1 SECT CARACOLES,PENUELAS,00624-0000,GLORIA GARCIA,YAUCO,MIGUEL GONZALEZ BAUZA,15.03,...,17.65,16.99,29.41,280,PEÑUELAS,ELEMENTAL,PONCE,759,836-2597,RURAL
2,65.31,23218,PR,URB REPARTO MONTEBANCO CALLE D,CAYEY,00737-0000,WANDA I.CARTAGENA ORTIZ,CIDRA,MIGUEL MELENDEZ MUNOZ,32.84,...,10.45,21.89,50.25,278,CAYEY,ELEMENTAL,CAGUAS,760,738-2058,URBANA
3,65.71,23531,PR,AVE ANTONIO R BARCELO FRENTE CUC,CAYEY,00737-0000,PEDRO SANCHEZ,CIDRA,MIGUEL MELENDEZ MUNOZ,23.98,...,10.40,17.34,30.06,762,CAYEY,SUPERIOR,CAGUAS,761,738-3078,URBANA
4,63.10,70276,PR,CARR 840 KM 1 HM 1 SECTOR LA ALDEA,BAYAMON,00960-3169,MARIA JUDITH VAZQUEZ FONTANEZ,BAYAMON,MIGUEL MELENDEZ MUNOZ,44.59,...,19.59,16.89,35.14,577,BAYAMON II,SUPERIOR,BAYAMON,762,799-7300,RURAL


## Loading Socioeconomic_Indicators_By_Municipality.csv into database censusData.sqlite

In [196]:
strCsvName = "Socioeconomic_Indicators_By_Municipality.csv"
strCsv_path = strCsvFolderPath + strCsvName
df_SIBy_Municipality = pd.read_csv(strCsv_path, encoding ='latin1')

In [197]:
df_SIBy_Municipality.head(15)

,Municipalities,Population 2010 (Census),"Unemployment rate (Department of Labor, December, 2012)","% Population 25+, with a bachelor degree or higher (ACS 2011)","Median household income (ACS, 2011)","Median Family Income (ACS, 2011)"
0,Adjuntas,"19,483",23.6,14.5,12975,16495
1,Aguada,"41,959",14.5,18.0,14935,16885
2,Aguadilla,"60,949",13.1,19.8,14779,18176
3,Aguas Buenas,"28,659",14.8,15.2,13792,16416
4,Aibonito,"25,900",15.8,20.1,15736,19336
5,Aasco,"29,261",15.0,14.9,16291,18910
6,Arecibo,"96,440",13.9,20.1,17089,20223
7,Arroyo,"19,575",18.0,14.8,15551,17511
8,Barceloneta,"24,816",14.2,16.9,16278,18341
9,Barranquitas,"30,318",16.0,16.7,15117,15996


In [198]:
del df_SIBy_Municipality['Municipalities']

In [199]:
df_SIBy_Municipality.head()

,Population 2010 (Census),"Unemployment rate (Department of Labor, December, 2012)","% Population 25+, with a bachelor degree or higher (ACS 2011)","Median household income (ACS, 2011)","Median Family Income (ACS, 2011)"
0,"19,483",23.6,14.5,12975,16495
1,"41,959",14.5,18.0,14935,16885
2,"60,949",13.1,19.8,14779,18176
3,"28,659",14.8,15.2,13792,16416
4,"25,900",15.8,20.1,15736,19336


In [200]:
l_cities = ["Adjuntas","Aguada","Aguadilla","Aguas Buenas","Aibonito","Añasco","Arecibo","Arroyo","Barceloneta","Barranquitas",
"Bayamon","Cabo Rojo","Caguas","Camuy","Canovanas","Carolina","Cataño","Cayey","Ceiba","Ciales","Cidra","Coamo","Comerio",
"Corozal","Culebra","Dorado","Fajardo","Florida","Guanica","Guayama","Guayanilla","Guaynabo","Gurabo","Hatillo","Hormigueros","Humacao",
"Isabela","Jayuya","Juana Díaz","Juncos","Lajas","Lares","Las Marías","Las Piedras","Loiza","Luquillo","Manati","Maricao",
"Maunabo","Mayagüez","Moca","Morovis","Naguabo","Naranjito","Orocovis","Patillas","Peñuelas","Ponce","Quebradillas","Rincon",
"Río Grande","Sabana Grande","Salinas","San German","San Juan","San Lorenzo","San Sebastián","Santa Isabel","Toa Alta","Toa Baja"
"Trujillo Alto","Utuado","Vega Alta","Vega Baja","Vieques","Villalba","Yabucoa","Yauco"]

In [201]:
df_Municipalities = pd.DataFrame( {'Municipalities':l_cities} )

In [202]:
df_Municipalities.head()

,Municipalities
0,Adjuntas
1,Aguada
2,Aguadilla
3,Aguas Buenas
4,Aibonito


In [203]:
df_SIBy_Municipality.insert(0,"Municipalities",df_Municipalities['Municipalities'])

In [204]:
df_SIBy_Municipality.head(20)

,Municipalities,Population 2010 (Census),"Unemployment rate (Department of Labor, December, 2012)","% Population 25+, with a bachelor degree or higher (ACS 2011)","Median household income (ACS, 2011)","Median Family Income (ACS, 2011)"
0,Adjuntas,"19,483",23.6,14.5,12975,16495
1,Aguada,"41,959",14.5,18.0,14935,16885
2,Aguadilla,"60,949",13.1,19.8,14779,18176
3,Aguas Buenas,"28,659",14.8,15.2,13792,16416
4,Aibonito,"25,900",15.8,20.1,15736,19336
5,Añasco,"29,261",15.0,14.9,16291,18910
6,Arecibo,"96,440",13.9,20.1,17089,20223
7,Arroyo,"19,575",18.0,14.8,15551,17511
8,Barceloneta,"24,816",14.2,16.9,16278,18341
9,Barranquitas,"30,318",16.0,16.7,15117,15996


## Upload to SQLite

In [205]:
database_path = strDatabaseFolderPath + strDataBaseName
engine = create_engine(f'sqlite:///{database_path}')

In [206]:
df_SIBy_Municipality.to_sql(name='tblSocioEcIndicators', con=engine, if_exists='append', index=False)

In [207]:
engine.table_names()

['tblMasterData', 'tblSocioEcIndicators']

In [209]:
pd.read_sql_query('select * from tblSocioEcIndicators LIMIT 5', con=engine)

,Municipalities,Population 2010 (Census),"Unemployment rate (Department of Labor, December, 2012)","% Population 25+, with a bachelor degree or higher (ACS 2011)","Median household income (ACS, 2011)","Median Family Income (ACS, 2011)"
0,Adjuntas,"19,483",23.6,14.5,12975,16495
1,Aguada,"41,959",14.5,18.0,14935,16885
2,Aguadilla,"60,949",13.1,19.8,14779,18176
3,Aguas Buenas,"28,659",14.8,15.2,13792,16416
4,Aibonito,"25,900",15.8,20.1,15736,19336
